# 중간고사 4번: 따릉이 수요 예측
*Linear Regression*

- CUDA 사용
- 전체 행 normalize
- mse 내장함수 사용

중간고사에서는 전체 행을 Normalize 하지 않고 std가 유난히 큰 hour_bef_visibility만 normalize했는데, 이번에는 전부 다 했다. 그런데 결과는 중간고사 버전이 월등히 뛰어나다. 왜 그럴까??

In [0]:
from google.colab import files
files.upload()

In [0]:
! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c 2020-ai-exam-biker-revisit

In [0]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd
import numpy as np

In [0]:
torch.manual_seed(1)
device = torch.device('cuda')

# Load Data

In [0]:
xy_train = pd.read_csv('2020.AI.bike-train.csv', header=0, index_col=0)
x_test = pd.read_csv('2020.AI.bike-test.csv', header=0, index_col=0)

In [84]:
xy_train.describe()

,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,count
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000
mean,11.886000,16.829000,0.033000,2.464600,52.012000,1412.057000,0.038480,57.44900,30.244000,110.693000
std,6.707757,5.196688,0.178726,1.386726,20.288358,575.842011,0.019338,30.79558,14.553428,83.867506
min,0.000000,3.100000,0.000000,0.000000,7.000000,78.000000,0.003000,12.00000,9.000000,1.000000
25%,6.000000,13.100000,0.000000,1.400000,36.000000,910.500000,0.024000,36.00000,20.000000,37.000000
50%,12.000000,16.600000,0.000000,2.300000,51.000000,1577.000000,0.038000,52.00000,26.000000,100.000000
75%,18.000000,20.100000,0.000000,3.400000,68.250000,1991.000000,0.051250,70.00000,37.000000,153.000000
max,23.000000,30.000000,1.000000,8.000000,99.000000,2000.000000,0.106000,269.00000,90.000000,431.000000


In [0]:
norm_data = dict()
def get_norm_data(col):
    norm_data[col+'_min'] = xy_train[col].min()
    norm_data[col+'_max'] = xy_train[col].max()
    norm_data[col+'_diff'] = norm_data[col+'_max'] - norm_data[col+'_min']
def normalizer(col, target):
    target[col] -= norm_data[col+'_min']
    target[col] /= norm_data[col+'_diff']

In [86]:
cols = xy_train.columns
test_cols = x_test.columns
cols

Index(['hour', 'hour_bef_temperature', 'hour_bef_precipitation',
       'hour_bef_windspeed', 'hour_bef_humidity', 'hour_bef_visibility',
       'hour_bef_ozone', 'hour_bef_pm10', 'hour_bef_pm2.5', 'count'],
      dtype='object')

In [0]:
# Data Normalization
for col in cols:
    get_norm_data(col)
    normalizer(col, xy_train)
for col in test_cols:
    normalizer(col, x_test)

In [88]:
xy_train.describe()

,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,count
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.516783,0.510372,0.033000,0.308075,0.489261,0.694098,0.344466,0.176844,0.262272,0.255100
std,0.291642,0.193185,0.178726,0.173341,0.220526,0.299606,0.187751,0.119827,0.179672,0.195041
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.260870,0.371747,0.000000,0.175000,0.315217,0.433143,0.203883,0.093385,0.135802,0.083721
50%,0.521739,0.501859,0.000000,0.287500,0.478261,0.779917,0.339806,0.155642,0.209877,0.230233
75%,0.782609,0.631970,0.000000,0.425000,0.665761,0.995317,0.468447,0.225681,0.345679,0.353488
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [95]:
x_train = np.array(xy_train.iloc[:, :9])
y_train = np.array(xy_train.iloc[:, 9])
x_test = np.array(x_test)

x_train = torch.FloatTensor(x_train).to(device)  # CUDA 연결
y_train = torch.FloatTensor(y_train).to(device)  # CUDA 연결
x_test = torch.FloatTensor(x_test).to(device)    # CUDA 연결

print(x_train.shape)
print(y_train.shape)

torch.Size([1000, 9])
torch.Size([1000])


# Train Model

In [119]:
Epochs = 5000
lr = 1e-5

W = torch.zeros((9, 1), requires_grad=True, device='cuda') # CUDA 연결
b = torch.zeros(1, requires_grad=True, device='cuda')      # CUDA 연결

optimizer = optim.SGD((W, b), lr=lr, momentum=0.9) #Implement Momentum

for epoch in range(1, Epochs+1):
    hypothesis = x_train.mm(W) + b
    cost = F.mse_loss(hypothesis, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch == 1 or epoch%1000 == 0:
        print('Epoch: {:4d}/{} Cost: {:.6f}'.format(epoch, Epochs, cost))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Using a target size (torch.Size([1000])) that is different to the input size (torch.Size([1000, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  # This is added back by InteractiveShellApp.init_path()


Epoch:    1/5000 Cost: 0.103079
Epoch: 1000/5000 Cost: 0.061711
Epoch: 2000/5000 Cost: 0.046909
Epoch: 3000/5000 Cost: 0.041684
Epoch: 4000/5000 Cost: 0.039826
Epoch: 5000/5000 Cost: 0.039152


# Test

In [108]:
pred = x_test.mm(W) + b
pred *= norm_data['count_diff']
pred += norm_data['count_min']
pred[:5]

tensor([[101.4925],
        [125.8270],
        [109.5320],
        [ 92.9402],
        [ 90.3737]], device='cuda:0', grad_fn=<SliceBackward>)

In [97]:
submit = pd.read_csv('2020.AI.bike-submission.csv')
submit['id'] = submit['count']

for i in range(len(submit)):
    submit['count'][i] = pred[i]

submit['id'] = pd.Series(submit['id'], dtype="Int32")
submit.head()

,id,count
0,0,101.502716
1,1,125.843536
2,2,109.545052
3,3,92.947540
4,4,90.380898


In [0]:
submit.to_csv('submit.csv', index=False)

In [116]:
! kaggle competitions submit -c 2020-ai-exam-biker-revisit -f submit.csv -m 'First Submit'

100% 7.03k/7.03k [00:06<00:00, 1.05kB/s]
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python2.7/dist-packages/kaggle/cli.py", line 64, in main
    print(out, end='')
UnicodeEncodeError: 'latin-1' codec can't encode characters in position 34-37: ordinal not in range(256)
